In [4]:
import os
os.getcwd()

'D:\\Python\\finominaScraping'

In [5]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import numpy as np

In [6]:
def transformSTG_X(path:str,QUARTER_REF:str=5):
    
    DF_RWST = pd.read_csv(path)
    DF_RWST = DF_RWST.rename(columns={
        'Unnamed: 0'            : "Date", 
        'รายได้รวม'              : "TotalRevenue", 
        'การเติบโตต่อไตรมาส (%)'  : "growth/quarter",
        'การเติบโตต่อปี (%)'       : "growth/year",
        'กำไรสุทธิ'               : "NetProfit",
        'กำไรต่อหุ้น (EPS)'        : "EPS",
        'ROA (%)'               : "ROA",
        'ROE (%)'               : "ROE",
        'อัตรากำไรขั้นต้น (%)'      : "GrossProfitMargin",
        'อัตราส่วนการขายและบริหารต่อรายได้ (%)'  : "OperatingMargin",
        'อัตรากำไรสุทธิ (%)'        : "NetProfitMargin",
        'หนี้สิน/ทุน (เท่า)'         : "debt/capital",
        'วงจรเงินสด (วัน)'          : "CashConversionCycle",
        'มูลค่าหลักทรัพย์ตามราคาตลาด (ล้านบาท)'  : "MKT.values",
        'P/E (เท่า)'                : "P/E",
        'P/BV (เท่า)'               : "P/BV",
        'มูลค่าหุ้นทางบัญชีต่อหุ้น (บาท)' : "BookValue/Share",
        'อัตราส่วนเงินปันผลตอบแทน'    : "DividendYield",
        'EV / EBITDA'              : "EV/EBITDA",
        'ราคาล่าสุด (บาท)'           : "Price",
    })
    DF_RWST['year'] = DF_RWST['Date'].str.slice(start=2)
    DF_RWST['quarter'] = DF_RWST['Date'].str.slice(stop=1)
    DF_RWST = DF_RWST.sort_values(['year','quarter'],ascending=False)


    DF_Cross = DF_RWST[ ["Symbol","Market","IndusGroup","BusinessGroup"] ]
    
    ##### transformDate #####   
    DF_Date     = pd.DataFrame()
    DF_Date['year']   = DF_RWST['year']
    DF_Date['quarter']= DF_RWST['quarter']
    
    #--tran พศ to คศ  
    year = DF_RWST['year'].to_numpy()
    year = year.astype(int) - 543
    DF_Date['Date'] = np.add(   
        np.add(DF_RWST['quarter'].astype(str),["Q"]),
        year.astype(str) )

    
    
    DF_Ts    = DF_RWST.drop( columns=list(DF_Cross.columns)+list(DF_Date.columns) )
    DF_Ts    = DF_Ts.reset_index(drop=True)
    
    ##### transformDF #####
    STG_symbol = pd.DataFrame()
    for i,c1 in DF_Ts.iterrows():
        transformDF = pd.DataFrame([c1.values], columns=c1.index).add_suffix(f'[0]') 
        ref = 0
        for j,c2 in DF_Ts[i+1:i+1+QUARTER_REF].iterrows():
            ref += 1
            transformDF = pd.concat([transformDF,pd.DataFrame([c2.values], columns=c2.index).add_suffix(f'[-{ref}]')  ] , axis=1)

        STG_symbol = pd.concat([STG_symbol,transformDF], axis=0) 
    STG_symbol = STG_symbol.reset_index(drop=True)
    
    # ADD "year","quarter"
    # STG_symbol = pd.concat([STG_symbol,DF_Date[ ["year","quarter"] ]   ], axis=1) 

    # ADD CrossSection data
    STG_X = pd.concat([STG_symbol,DF_Cross   ], axis=1) 
    
    DF_Date = DF_Date.reset_index()
    STG_X['Date'] = DF_Date['Date']
    
    return STG_X.set_index('Date') ,DF_Date.set_index('Date')
    

In [7]:
def transformSTG_Y(STG_X,Date):
    Y={
        'Price[1]':[np.nan]*1, #1q
        'Price[2]':[np.nan]*2, #2q
        'Price[3]':[np.nan]*3, #3q
        'Price[4]':[np.nan]*4 #4q
    }

    price = STG_X['Price[0]'].values

    for i in range(len(STG_X)):
        for j in range( 1,len(Y)+1 ):
            if i >= j:
                Y[f'Price[{j}]'] += [price[i-j]]    
                
    Y = pd.DataFrame(Y)
    Y['Price[0]'] = STG_X['Price[0]'].values
    Y = Y[Y.columns.sort_values()]
    Y['Date'] = Date.index
    return Y.set_index("Date")

In [12]:
def ratingScore(STG_Y,Date):
    STG_Y['naValue']   = np.any(  [pd.isnull(STG_Y[c]).values  for c in  STG_Y.columns] ,axis = 0 )
    STG_Y['score']     = np.nan
    columns = STG_Y.columns
    
    for i,r in STG_Y.iterrows():
        if r['naValue'] == True:
            r['score']  = np.nan
        else:
            if r["Price[0]"] == 0 :
                r['score'] = np.nan
            else:
                diff = (r["Price[1]"] / r["Price[0]"])-1
                if diff >= 0.2 :
                    r['score'] = 3
                elif diff >= 0.1 :
                    r['score'] = 2
                elif diff >= 0 :
                    r['score'] = 1
                else:
                    r['score'] = 0
    return STG_Y

In [9]:
STG_X,Date = transformSTG_X('./data/RWST/MFC.csv',)

In [12]:
STG_Y = transformSTG_Y(STG_X,Date)

In [14]:
len(STG_X)

43

In [106]:
#get set index
SET = pd.read_csv('./data/SET.BK.csv')


In [64]:
dateSplit = pd.DataFrame()
dateSplit[['day','quarter','year']]=SET['Timestamp'].str.split("-",n=0,expand=True)

In [71]:
dateSplit['quarter']=dateSplit['quarter'].replace("01","4")
dateSplit['quarter']=dateSplit['quarter'].replace("04","1")
dateSplit['quarter']=dateSplit['quarter'].replace("07","2")
dateSplit['quarter']=dateSplit['quarter'].replace("10","3")

In [92]:
dateSplit['year'] = pd.to_datetime(dateSplit['year'].str.strip().astype(str), format='%y').dt.strftime('%Y')

In [98]:
dateSplit['Date'] = np.add(   
    np.add(dateSplit['quarter'].astype(str),["Q"]),
    dateSplit['year'].astype(str) )

In [108]:
dateSplit['Date'].sort_values('',ascending=False)

ValueError: No axis named  for object type Series

In [77]:
dateSplit['year'].str.strip().strftime('%Y')

AttributeError: 'Series' object has no attribute 'strftime'

In [53]:
Y = pd.DataFrame( {"Date":q[1].str.strip()} )

In [54]:
Y

,Date
0,04-75
1,07-75
2,10-75
3,01-76
4,04-76
...,...
186,10-21
187,01-22
188,04-22
189,07-22


In [13]:
ratingScore(STG_Y,Date)

21.9
19.0
17.1
14.6
13.6
13.4
12.9
15.4
17.3
18.0
18.6
16.7
16.9
15.0
17.4
16.2
20.0
20.3
22.9
21.9
22.3
22.2
24.3
24.7
0.0


ZeroDivisionError: float division by zero

In [11]:
len(STG_X)

43

In [ ]:
STG_Y.head(3)

In [ ]:
STG_X.head(3)